## Initial work
Modified the part that sets the folder paths, etc. (changed to the methods used in the PSR_MVPA_crossvalidation_fix.m file from the included .zip).

In [ ]:
% 10-fold cross-validation and time generalization within each condition
%   MEG data is aligned to S1 for the all conditions, additionally, data
%   are aligned to saccade onset and offset for the saccade condition

%% Init                         
% initialize
clearvars; close all; clc;

% add paths to functions
PSR_setpaths;

% set folders
folder.IN   = fullfile(cd,'TIMELOCKED','S1');
folder.TRL  = fullfile(cd,'TRIALS','REJECTVISUAL');
folder.INFO = fullfile(cd,'TRIALS','INFO');
folder.OUT  = cd;

% subject
subjects = PSR_subjects(folder.IN);

## Parameters
Below, the parameters are given, with their default values. This cell is tagged with the tagg "parameters". Papermill would inject a cell with parameters below this one, overriding the defaults with whatever setting you want.

In [ ]:
%% Parameters                   
p.dsfs       = 250;         % downsampled frequency
p.bws        = -0.2;        % baseline window start
p.bwe        = 0;           % baseline window end
p.mintime    = -0.1;        % do not use the whole eocht from -0.5 to 1.5, but start at [mintime]
p.maxtime    = 0.9;         % do not use the whole eocht from -0.5 to 1.5, but end at [maxtime]
p.saccut     = [-0.6 0.4];  % saccade epoch
p.timeradius = 2;           % for temporal 'searchlight'
suffix       = {'LEFTS1','RIGHTS1','SACS1','SACON','SACOFF'};

## Core code
Left unchanged. Suggestion: consider throwing an error (or at least printing some message) if the run is aborted because the desired output file already exists.

In [ ]:
% CoSMo measure argurments
p.measure_args            = struct();
p.measure_args.measure    = @cosmo_crossvalidation_measure;
p.measure_args.classifier = @cosmo_classify_svm;
p.measure_args.dimension  = 'time';
p.measure_args.radius     = p.timeradius;
p.measure_args.output     = 'accuracy';   
p.measure_args.nproc      = 12;

%% Loop over subjects           
for isubj = 1:numel(subjects)
    %% Print message                        
    fprintf('\n%s\n',subjects{isubj});
    files = dir(folder.OUT);
    files = cat(1,{files.name});
    if sum(contains(files,subjects{isubj})) == 5 || ...
       strcmp(subjects{isubj},'19820525DNCC')
        continue
    end
    
    %% Load dataset                         
    tic;
    fprintf('Loading dataset... ');
    
    channels = 'MEG';
    [data] = PSR_loadTimelock(folder.IN,folder.TRL,'S1',subjects{isubj},channels);
    dataS1 = data.s1lock;
    dataS1 = rmfield(dataS1, 'cfg');
    dataS1 = ft_struct2double(dataS1);
    
    clear data
    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% Downsample                           
    tic;
    fprintf('Downsampling data to %d Hz... ',p.dsfs);
    
    dataS1.fsample = 500;
    cfg            = [];
    cfg.resamplefs = p.dsfs;
    dataS1         = ft_resampledata(cfg,dataS1);
    dataS1.time    = cellfun(@(x) round(x*1000)/1000,dataS1.time,'UniformOutput',false);
    
    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% Standardize data                     
    tic;
    fprintf('Standardizing data... ');
    
    % 1. baseline correction
    cfg = [];
    cfg.demean = 'yes';
    cfg.baselinewindow = [p.bws p.bwe];
    [dataS1] = ft_preprocessing(cfg,dataS1);
    
    % 2. standardize data per sensor
    i1 = find( abs(dataS1.time{1}-p.bws) == min(abs(dataS1.time{1}-p.bws)) );
    i2 = find( abs(dataS1.time{1}-p.bwe) == min(abs(dataS1.time{1}-p.bwe)) );
    bw_std = std( cell2mat( cellfun(@(x) x(:,i1:i2),dataS1.trial,'UniformOutput',false) ), [], 2);
    dataS1.trial = cellfun(@(x) bsxfun(@rdivide,x,bw_std), dataS1.trial,'UniformOutput',false);

    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: S1 onset aligned         
    tic;
    fprintf('\nPreparing CoSMO data, aligned to S1...\n');
    for k = 1:3
        % 1. select conditions
        cfg = [];
        switch k
            case 1
                cfg.trials = PSR_selectCondition(dataS1,{'fll','flh'});
            case 2
                cfg.trials = PSR_selectCondition(dataS1,{'frl','frh'});
            case 3
                cfg.trials = PSR_selectCondition(dataS1,{'srl','srh'});
        end
        [dataTMP] = ft_selectdata(cfg,dataS1);
        
        % 2. make CoSMoMVPA data
        ds.(suffix{k}) = PSR_mkCoSMoData(dataTMP);
        clear dataTMP
        
        % 3. make epoch smaller (from [-0.5,1.5] to [mintime,maxtime])
        i1 = find(ds.(suffix{k}).a.fdim.values{2}==p.mintime) - p.timeradius;
        i2 = find(ds.(suffix{k}).a.fdim.values{2}==p.maxtime) + p.timeradius;
        timemask = ds.(suffix{k}).fa.time >=i1 & ds.(suffix{k}).fa.time <= i2;
        ds.(suffix{k}) = cosmo_slice(ds.(suffix{k}), timemask, 2);
        ds.(suffix{k}) = cosmo_dim_prune(ds.(suffix{k}));
        cosmo_check_dataset(ds.(suffix{k}));
        
    end
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: saccade onset aligned    
    tic;
    fprintf('\nPreparing CoSMO data, aligned to saccade onset...\n');
    
    % 1. SACCADE data: cut and align epochs to saccade onset
    [dataTMP] = PSR_cutsaconset( folder.INFO, dataS1, subjects{isubj}, 'SAC', p.saccut+[-p.timeradius/p.dsfs p.timeradius/p.dsfs]);
    
    % 2. select conditions for SACCADE data
    cfg            = [];
    cfg.trials     = PSR_selectCondition(dataTMP,{'srl','srh'});
    dataTMP        = ft_selectdata(cfg,dataTMP);
    ds.(suffix{4}) = PSR_mkCoSMoData(dataTMP);
    clear dataTMP
    
    % 3. remove irrelevant field
    ds.(suffix{4}).sa = rmfield(ds.(suffix{4}).sa,'isrt');
    
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: saccade offset aligned   
    tic;
    fprintf('\nPreparing CoSMO data, aligned to saccade offset...\n');
    
    % 1. SACCADE data: cut and align epochs to saccade onset
    [dataTMP] = PSR_cutsaconset( folder.INFO, dataS1, subjects{isubj}, 'SACOFF', p.saccut+[-p.timeradius/p.dsfs p.timeradius/p.dsfs]);
    
    % 2. select conditions for SACCADE data
    cfg            = [];
    cfg.trials     = PSR_selectCondition(dataTMP,{'srl','srh'});
    dataTMP        = ft_selectdata(cfg,dataTMP);
    ds.(suffix{5}) = PSR_mkCoSMoData(dataTMP);

    % 3. remove irrelevant field
    ds.(suffix{5}).sa = rmfield(ds.(suffix{5}).sa,'isrt');

    % remove large redundant variables
    clear dataS1 dataTMP
    
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% MVPA                                 
    for s = 1:length(suffix)
        % print message                    
        tstart = tic;
        fprintf('Classification %s\n', suffix{s} );
        fileOUT = [folder.OUT filesep subjects{isubj} '_' suffix{s} '.mat'];
        if exist(fileOUT,'file')
            continue;
        end
        
        % select data
        dsTMP = ds.(suffix{s});
        dsTMP.sa.targets = dsTMP.sa.sf;
        
        % set chunks (split half for cosmo_dim_generalization_measure)
        TMP.sf1    = find(dsTMP.sa.targets==1);
        TMP.sf2    = find(dsTMP.sa.targets==2);
        TMP.chunk1 = sort([TMP.sf1(1:2:end); TMP.sf2(1:2:end)]);
        TMP.chunk2 = sort([TMP.sf1(2:2:end); TMP.sf2(2:2:end)]);
        dsTMP.sa.chunks(TMP.chunk1) = 1;
        dsTMP.sa.chunks(TMP.chunk2) = 2;
        clearvars TMP
        
                
        % 4. balance trial types per half
        dsTMP1 = cosmo_slice(dsTMP,dsTMP.sa.chunks==1);
        dsTMP2 = cosmo_slice(dsTMP,dsTMP.sa.chunks==2);
        if ismember( subjects{isubj}, {'19910219ANSL', '19910228FLPE'} )
            dsTMP1 = PSR_balanceCoSMoTrialtype(dsTMP1,'sfori');
            dsTMP2 = PSR_balanceCoSMoTrialtype(dsTMP2,'sfori');
        else
            dsTMP1 = PSR_balanceCoSMoTrialtype(dsTMP1);
            dsTMP2 = PSR_balanceCoSMoTrialtype(dsTMP2);
        end
        dsTMP = cosmo_stack({dsTMP1, dsTMP2});
        cosmo_check_dataset(dsTMP);
        clearvars dsTMP1 dsTMP2
        
        % transpose time to the sample dimension
        dsTMP = cosmo_dim_transpose(dsTMP,'time',1);
        
        % run temporal generalization
        result = cosmo_dim_generalization_measure(dsTMP, p.measure_args, ...
                                                  'dimension', 'time', ...
                                                  'radius', p.timeradius, ...
                                                  'nproc',12);
        
        % transpose time back
        val = result.a.sdim.values;
        lbl = result.a.sdim.labels;
        acc = zeros(length(val{1}),length(val{2}));
        for j = 1:length(val{1})
            for k = 1:length(val{2})
                select   = result.sa.(lbl{1})==j & result.sa.(lbl{2})==k;
                acc(j,k) = result.samples(select);
            end
        end
                                              
        % save data
        save(fileOUT,'result','p','acc');
        
        % clear redundant variables
        clearvars dsTMP result lbl acc select
        
    end
    
    % remove large, redundant variables
    clearvars -except folder p subjects suffix
    
end